# Alternative Models

### Set up

In [41]:
### Load necessary libraries ###
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, Input
from keras.models import Model

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from librosa import display
import librosa

In [6]:
x_train = pd.read_csv("allFeaturesData/train_data.csv").to_numpy().reshape(7894, 40, 5, 1)
x_test = pd.read_csv("allFeaturesData/test_data.csv").to_numpy().reshape(836, 40, 5, 1)
y_train = pd.read_csv("allFeaturesData/train_labels.csv").to_numpy().reshape(7894,)
y_test = pd.read_csv("allFeaturesData/test_labels.csv").to_numpy().reshape(836,)

In [13]:
pool_size = (2, 2)
kernel_size = (3, 3)
input_shape = (40, 5, 1)
num_classes = 10

### LeNet-5

In [82]:
# model
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(40,5,1),padding='same'))
model.add(keras.layers.AveragePooling2D(padding='same'))
model.add(keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.AveragePooling2D(padding='same'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=120, activation='relu'))
model.add(keras.layers.Dense(units=84, activation='relu'))
model.add(keras.layers.Dense(units=10, activation = 'softmax'))


optimizer = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

C:\Users\EmmaC\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [83]:
keras.backend.clear_session()
model.fit(x_train, y_train, epochs = 50, batch_size = 50, validation_data = (x_test, y_test))

Epoch 1/50
158/158 [==============================] - 1s 2ms/step - loss: 2.1430 - accuracy: 0.2468 - val_loss: 1.9677 - val_accuracy: 0.3624
Epoch 2/50
158/158 [==============================] - 0s 2ms/step - loss: 1.8360 - accuracy: 0.4187 - val_loss: 1.6932 - val_accuracy: 0.4928
Epoch 3/50
158/158 [==============================] - 0s 2ms/step - loss: 1.5864 - accuracy: 0.4799 - val_loss: 1.5350 - val_accuracy: 0.5610
Epoch 4/50
158/158 [==============================] - 0s 2ms/step - loss: 1.4282 - accuracy: 0.5206 - val_loss: 1.4356 - val_accuracy: 0.5885
Epoch 5/50
158/158 [==============================] - 0s 2ms/step - loss: 1.3253 - accuracy: 0.5523 - val_loss: 1.3963 - val_accuracy: 0.5646
Epoch 6/50
158/158 [==============================] - 0s 2ms/step - loss: 1.2477 - accuracy: 0.5752 - val_loss: 1.3528 - val_accuracy: 0.5885
Epoch 7/50
158/158 [==============================] - 0s 2ms/step - loss: 1.1869 - accuracy: 0.6005 - val_loss: 1.3215 - val_accuracy: 0.5945
Epoch 

In [80]:
predictions = model.predict(x_test)
predictions = np.apply_along_axis(lambda x : list(x).index(max(x)),1, predictions)
predictions.shape

27/27 [==============================] - 0s 731us/step


(836,)

In [81]:
pd.DataFrame({
    "prediction" : predictions,
    "actual" : y_test
}).groupby("actual") \
    .apply(lambda x:x["prediction"]==x["actual"]) \
    .groupby("actual") \
    .agg(Accuracy = 'mean') \
    .reset_index() \
    .assign(names=[
        "air_conditioner",
        "car_horn",
        "children_playing",
        "dog_bark",
        "drilling",
        "engine_idling",
        "gun_shot",
        "jackhammer",
        "siren",
        "street_music"
    ])

,actual,Accuracy,names
0,0.0,0.590000,air_conditioner
1,1.0,0.718750,car_horn
2,2.0,0.720000,children_playing
3,3.0,0.610000,dog_bark
4,4.0,0.710000,drilling
5,5.0,0.720430,engine_idling
6,6.0,0.687500,gun_shot
7,7.0,0.395833,jackhammer
8,8.0,0.493976,siren
9,9.0,0.740000,street_music


### GoogleNet

In [34]:
input_shape = Input(shape=(40, 5, 1))

In [37]:
### 1st layer
layer_1 = Conv2D(10, (1,1), padding='same', activation='relu')(input_shape)
layer_1 = Conv2D(10, (3,3), padding='same', activation='relu')(layer_1)

layer_2 = Conv2D(10, (1,1), padding='same', activation='relu')(input_shape)
layer_2 = Conv2D(10, (5,5), padding='same', activation='relu')(layer_2)

layer_3 = MaxPool2D((3,3), strides=(1,1), padding='same')(input_shape)
layer_3 = Conv2D(10, (1,1), padding='same', activation='relu')(layer_3)

mid_1 = keras.layers.concatenate([layer_1, layer_2, layer_3], axis = 3)

In [39]:
flat_1 = Flatten()(mid_1)

dense_1 = Dense(1200, activation='relu')(flat_1)
dense_2 = Dense(600, activation='relu')(dense_1)
dense_3 = Dense(150, activation='relu')(dense_2)
output = Dense(num_classes, activation='softmax')(dense_3)

In [43]:
model = Model([input_shape], output)

In [44]:
optimizer = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

C:\Users\EmmaC\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [45]:
keras.backend.clear_session()
model.fit(x_train, y_train, epochs = 50, batch_size = 50, validation_data = (x_test, y_test))

Epoch 1/50
158/158 [==============================] - 6s 36ms/step - loss: 1.4690 - accuracy: 0.5438 - val_loss: 1.3854 - val_accuracy: 0.5060
Epoch 2/50
158/158 [==============================] - 6s 37ms/step - loss: 0.8349 - accuracy: 0.7299 - val_loss: 1.0893 - val_accuracy: 0.6878
Epoch 3/50
158/158 [==============================] - 6s 37ms/step - loss: 0.5936 - accuracy: 0.8173 - val_loss: 1.1013 - val_accuracy: 0.6842
Epoch 4/50
158/158 [==============================] - 6s 36ms/step - loss: 0.4463 - accuracy: 0.8661 - val_loss: 1.1497 - val_accuracy: 0.6854
Epoch 5/50
158/158 [==============================] - 6s 36ms/step - loss: 0.3801 - accuracy: 0.8817 - val_loss: 1.0787 - val_accuracy: 0.6962
Epoch 6/50
158/158 [==============================] - 6s 37ms/step - loss: 0.3319 - accuracy: 0.8999 - val_loss: 1.1257 - val_accuracy: 0.7153
Epoch 7/50
158/158 [==============================] - 6s 36ms/step - loss: 0.2899 - accuracy: 0.9151 - val_loss: 1.3297 - val_accuracy: 0.6400

In [46]:
predictions = model.predict(x_test)
predictions = np.apply_along_axis(lambda x : list(x).index(max(x)),1, predictions)
predictions.shape

27/27 [==============================] - 0s 5ms/step


(836,)

In [47]:
pd.DataFrame({
    "prediction" : predictions,
    "actual" : y_test
}).groupby("actual") \
    .apply(lambda x:x["prediction"]==x["actual"]) \
    .groupby("actual") \
    .agg(Accuracy = 'mean') \
    .reset_index() \
    .assign(names=[
        "air_conditioner",
        "car_horn",
        "children_playing",
        "dog_bark",
        "drilling",
        "engine_idling",
        "gun_shot",
        "jackhammer",
        "siren",
        "street_music"
    ])

,actual,Accuracy,names
0,0.0,0.490000,air_conditioner
1,1.0,0.812500,car_horn
2,2.0,0.640000,children_playing
3,3.0,0.830000,dog_bark
4,4.0,0.660000,drilling
5,5.0,0.623656,engine_idling
6,6.0,0.906250,gun_shot
7,7.0,0.666667,jackhammer
8,8.0,0.566265,siren
9,9.0,0.670000,street_music


### VGG-16

In [67]:
pool_size = (2, 2)
kernel_size = (3, 3)
input_shape = (40, 5, 1)
num_classes = 10

In [68]:
model = Sequential()
model.add(Conv2D(input_shape=(40,5,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding="same"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding="same"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1, activation="softmax"))

In [72]:
optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [73]:
keras.backend.clear_session()
model.fit(x_train, y_train, epochs = 50, batch_size = 50, validation_data = (x_test, y_test))

Epoch 1/50
158/158 [==============================] - 86s 543ms/step - loss: 0.0000e+00 - accuracy: 0.0502 - val_loss: 0.0000e+00 - val_accuracy: 0.0383
Epoch 2/50
158/158 [==============================] - 86s 545ms/step - loss: 0.0000e+00 - accuracy: 0.0502 - val_loss: 0.0000e+00 - val_accuracy: 0.0383
Epoch 3/50
158/158 [==============================] - 86s 545ms/step - loss: 0.0000e+00 - accuracy: 0.0502 - val_loss: 0.0000e+00 - val_accuracy: 0.0383
Epoch 4/50
158/158 [==============================] - 86s 545ms/step - loss: 0.0000e+00 - accuracy: 0.0502 - val_loss: 0.0000e+00 - val_accuracy: 0.0383
Epoch 5/50
158/158 [==============================] - 87s 548ms/step - loss: 0.0000e+00 - accuracy: 0.0502 - val_loss: 0.0000e+00 - val_accuracy: 0.0383
Epoch 6/50
158/158 [==============================] - 87s 552ms/step - loss: 0.0000e+00 - accuracy: 0.0502 - val_loss: 0.0000e+00 - val_accuracy: 0.0383
Epoch 7/50
158/158 [==============================] - 86s 547ms/step - loss: 0.000

In [74]:
predictions = model.predict(x_test)
predictions = np.apply_along_axis(lambda x : list(x).index(max(x)),1, predictions)
predictions.shape

27/27 [==============================] - 1s 24ms/step


ValueError: nan is not in list

In [ ]:
pd.DataFrame({
    "prediction" : predictions,
    "actual" : y_test
}).groupby("actual") \
    .apply(lambda x:x["prediction"]==x["actual"]) \
    .groupby("actual") \
    .agg(Accuracy = 'mean') \
    .reset_index() \
    .assign(names=[
        "air_conditioner",
        "car_horn",
        "children_playing",
        "dog_bark",
        "drilling",
        "engine_idling",
        "gun_shot",
        "jackhammer",
        "siren",
        "street_music"
    ])